In [1]:
import pandas as pd

# Read the CSV file with the specified encoding
df = pd.read_csv("Exchange_Rates.csv", encoding='ISO-8859-1')

# Display the data types of the columns
print(df.dtypes)


Date         object
Currency     object
Exchange    float64
dtype: object


In [2]:

date_formats = ['%d-%m-%Y', '%m/%d/%Y'] #date format available in my file

def parse_dates(date_str): 
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt) 
        except ValueError:
            continue
    return pd.NaT #print not a time

# Apply the parsing function to the 'date' column
df['Date'] = df['Date'].apply(parse_dates)

# Convert to the desired format
df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')



In [3]:
df.isnull().sum()

Date        0
Currency    0
Exchange    0
dtype: int64

In [4]:
df.to_csv('Cleaned_Exchange_Rates.csv', index=False)


In [5]:
import pymysql
import pandas as pd

# Connect to MySQL
myconnection = pymysql.connect(host='127.0.0.1', user='root', passwd='password', database="project")

# Load CSV data
df = pd.read_csv("Cleaned_Exchange_Rates.csv")
df = df.dropna()  # Drop null values

# Replace data types from pandas to MySQL
a = ",".join(f"`{i}` {j}" for i, j in zip(df.columns, df.dtypes)).replace("float64", "FLOAT").replace("object", "TEXT")

# Use a valid table name
table_name = "Exchange_Rates"

# Create the table
create_table_query = f"CREATE TABLE `{table_name}` ({a})"
with myconnection.cursor() as cursor:
    cursor.execute(create_table_query)

# Insert data into the table
for i in range(len(df)):
    insert_query = f"INSERT INTO `{table_name}` VALUES {tuple(df.iloc[i])}"
    with myconnection.cursor() as cursor:
        cursor.execute(insert_query)
    myconnection.commit()

myconnection.close()